In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapely.geometry
import requests
import geemap
import os
import seaborn as sns
import boto3
sns.set_theme(style="darkgrid")
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"
# aws_s3_dir = "https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data"

In [ ]:
# USER INPUTS

YEARS = range(2020, 2099)
startyear = 2020
endyear = 2050

startyearStr = str(startyear)
endyearStr = str(endyear)

LONLAT = (76.383896, 10.426311,) # (x, y) for Kerala

current_geom = ee.Geometry.Point(LONLAT)

In [ ]:
maxprecip_probs = ee.ImageCollection('users/tedwongwri/dataportal/posterior_probs/maxprecip')

In [ ]:
def expected_value(year, geom, probdata):
# Return expected value of hazard for given year
    probs = probdata.filterMetadata('year', 'equals', year).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=geom, scale=250, bestEffort=True).getInfo()
    if np.sum([float(i) for i in probs.values()]) == 0:
        return 0, 1
    dataprobs = np.array([float(i) for i in probs.values()]) / np.sum([float(i) for i in probs.values()])
    mag_range = [(j.split('-')[0], j.split('-')[1]) for j in [i.split('_')[1] for i in list(probs.keys())]]
    avg_mags = np.array([(float(i[0].replace('neg', '-')) + float(i[1].replace('neg', '-'))) / 2 for i in mag_range])
    ev = np.dot(dataprobs, avg_mags) 
    for idx in range(len(mag_range)):
        mag_pair = mag_range[idx]
        if ev >= float(mag_pair[0].replace('neg', '-')) and ev < float(mag_pair[1].replace('neg', '-')):
            break
    key = list(probs.keys())[idx]
    return ev

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
# boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

In [ ]:
this_indicatorDF = pd.DataFrame() 

In [ ]:
for i in range(0, len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
    print("\n geo_name: "+boundary_id_aoi)
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    for feature in boundary_geo['features']:
            # print("\n     geo_id: " + feature['properties']['geo_id'])
            shape = shapely.geometry.shape(feature['geometry'])
            centroid = shape.centroid
            clon, clat = centroid.coords[0]
            LONLAT = (clon, clat,)
            current_geom = ee.Geometry.Point(LONLAT)
            geo_id = feature['properties']['geo_id']
            # maxprecip_evs = [expected_value(y, current_geom, maxprecip_probs) for y in YEARS]
            maxprecip_startev = expected_value(startyear, current_geom, maxprecip_probs)
            maxprecip_endev = expected_value(endyear, current_geom, maxprecip_probs)
            pctChange = ((maxprecip_endev /maxprecip_startev) - 1) # * 100
            df = pd.DataFrame({'geo_id':geo_id, 'maxDailymmPrecipin'+endyearStr+'': maxprecip_endev, 'GRE_4_2_percentChangeinMaxDailyPrecip'+startyearStr+'to'+endyearStr+'':pctChange}, index=[0])
            this_indicatorDF = pd.concat([this_indicatorDF,df])

    # # process unit of analysis level ------
    # print("\n boundary_id_unit: "+boundary_id_unit)
    # # read boundaries
    # boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    # boundary_geo = requests.get(boundary_path).json()
    # boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    # for feature in boundary_geo['features']:
    #         # print("\n     geo_id: " + feature['properties']['geo_id'])
    #         shape = shapely.geometry.shape(feature['geometry'])
    #         centroid = shape.centroid
    #         clon, clat = centroid.coords[0]
    #         LONLAT = (clon, clat,)
    #         current_geom = ee.Geometry.Point(LONLAT)
    #         geo_id = feature['properties']['geo_id']
    #         # maxprecip_evs = [expected_value(y, current_geom, maxprecip_probs) for y in YEARS]
    #         maxprecip_startev = expected_value(startyear, current_geom, maxprecip_probs)
    #         maxprecip_endev = expected_value(endyear, current_geom, maxprecip_probs)
    #         pctChange = ((maxprecip_endev /maxprecip_startev) - 1) # * 100
    #         df = pd.DataFrame({'geo_id':geo_id, 'maxDailymmPrecipin'+endyearStr+'': maxprecip_endev, 'percentChangeinMaxDailyPrecip'+startyearStr+'to'+endyearStr+'':pctChange}, index=[0])
    #         DF = pd.concat([DF,df])
            

In [ ]:
this_indicatorDF

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2test.csv') 
cities_indicators#.head()

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicators")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicatorDF,
                                            indicator_name = 'GRE_4_2_percentChangeinMaxDailyPrecip'+startyearStr+'to'+endyearStr+'')

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2test.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')